In [ ]:
%load_ext autoreload
%autoreload 2

## PlanB Quant Investing 101

Python implementation of PlanB Quant Investing article: https://planbtc.com/20220807QuantInvesting101.pdf

https://twitter.com/100trillionUSD/status/1556626501692526597

In [ ]:
#import sys
#!{sys.executable} -m pip install numpy==1.21.5
#!{sys.executable} -m pip install pandas==1.3.5
#!{sys.executable} -m pip install plotly==4.14.3
#!{sys.executable} -m pip install bt==0.2.9
#!{sys.executable} -m pip install coinmetrics-api-client==2022.7.14.6

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from coinmetrics.api_client import CoinMetricsClient
import bt

### Gather price data

In [ ]:
# Obtain metrics by specifying the asset, date interval and frequency.
# All metrics available: https://docs.coinmetrics.io/info/metrics
client = CoinMetricsClient()
price = client.get_asset_metrics(
    assets='btc',
    metrics='PriceUSD',
    #start_time='2010-07-10',
    #end_time='2021-12-17',
    frequency='1d'
    )
price = pd.DataFrame(price)
price['time'] = pd.to_datetime(price['time'])
price = price.set_index('time')
price['PriceUSD'] = pd.to_numeric(price['PriceUSD'])

### Functions

In [ ]:
def get_planBTC_strategy(rsi_value, buy_first=True):
    """
    https://planbtc.com/20220807QuantInvesting101.pdf
    * Data tested from January 2011

    BTC monthly closing data.
    IF (RSI was above 90% last six months AND drops below 65%) THEN sell,
    IF (RSI was below 50% last six months AND jumps +2% from the low) THEN buy, 
    ELSE hold
    """
    rsi_value.columns = ['value']

    # strategy params
    months_range = 6
    overbought_value = 90
    overbought_drop_value = 65
    oversold_value = 50

    signal = rsi_value.copy()
    long = False
    if buy_first:
        signal.iloc[0] = 1.0
        long = True

    for i in range(rsi_value.size):
        rsi = rsi_value.iloc[i]
        max_rsi_last_6_months = rsi_value[i-months_range:i+1].max()
        min_rsi_last_6_months = rsi_value[i-months_range:i+1].min()
        if i < 6:
            max_rsi_last_6_months = rsi_value[:i+1].max()
            min_rsi_last_6_months = rsi_value[:i+1].min()

        # SELL
        if (max_rsi_last_6_months >= overbought_value and 
                rsi <= overbought_drop_value and 
                long):
            signal.iloc[i] = -1.0
            long = False
        
        # BUY
        if (min_rsi_last_6_months <= oversold_value and 
                rsi >= min_rsi_last_6_months + 2 and 
                not long):
            signal.iloc[i] = 1.0
            long = True

    signal[(signal != 1.0) & (signal != -1.0)] = 0.0

    return signal

def get_rsi(close_price, period = 14):
    """
    Calculate RSI value.
    """
    delta = close_price.diff()

    up = delta.copy()
    up[up < 0] = 0
    up = pd.Series.ewm(up, alpha=1/period).mean()

    down = delta.copy()
    down[down > 0] = 0
    down *= -1
    down = pd.Series.ewm(down, alpha=1/period).mean()

    rsi = np.where(up == 0, 0, np.where(down == 0, 100, 100 - (100 / (1 + up / down))))

    return np.round(rsi, 2)

def convert_signal_to_weight(signal):
    """
    Function to convert signal signals (1, -1) to bt weight.
    """
    return signal.replace(0, np.nan).replace(-1, 0).ffill().replace(np.nan, 0)

def buy_and_hold_strategy(price_data, name='benchmark'):
    # Define the benchmark strategy
    s = bt.Strategy(name, 
                    [bt.algos.RunOnce(),
                     bt.algos.SelectAll(),
                     bt.algos.WeighEqually(),
                     bt.algos.Rebalance()])
    # Return the backtest
    return bt.Backtest(s, price_data)

def signal_strategy(price_data, signal, name):
    weight = convert_signal_to_weight(signal.copy())

    # the column names must be the same
    price_data.columns = ['value']
    weight.columns = ['value']

    s = bt.Strategy(name,
                    [bt.algos.WeighTarget(weight), 
                     bt.algos.Rebalance()])

    return bt.Backtest(s, price_data)

### Run Strategy

In [ ]:
# PlanBTC RSI strategy

# Resample data to 1 month
df_planBTC = price.resample('1M').last()
# remove incomplete candle
df_planBTC = df_planBTC[:-1]

df_planBTC['RSI'] = get_rsi(df_planBTC['PriceUSD'])
df_planBTC = df_planBTC['2011-01-01':]
df_planBTC.dropna(inplace=True)

df_planBTC['Signal'] = get_planBTC_strategy(df_planBTC['RSI'].copy())

# backtesting
bt_plan_BTC = signal_strategy(df_planBTC[['PriceUSD']].copy(), df_planBTC[['Signal']].copy(), 'planBTC')
bt_buy_and_hold = buy_and_hold_strategy(df_planBTC[['PriceUSD']].copy(), name='buy_and_hold')
bt_results = bt.run(bt_buy_and_hold, bt_plan_BTC)

### Plot

In [ ]:
# Plot Strategy
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True,
    specs=[[{'secondary_y': True}],[{}]],
    vertical_spacing=0.01, 
    row_heights=[0.7, 0.3])

# Add traces
# Price
fig.add_trace(
    go.Scatter(x=df_planBTC.index, y=df_planBTC['PriceUSD'], name='Price', legendgroup = '1'),
        secondary_y=True)
# RSI
fig.add_trace(
    go.Scatter(x=df_planBTC.index, y=df_planBTC['RSI'], name='RSI', mode='markers+lines', line_color='purple', legendgroup = '1'),
        secondary_y=False)
# RSI range 50-90
fig.add_hrect(y0=50, y1=90, line_width=0, fillcolor='purple', opacity=0.2, secondary_y=False)
# Buy signal
fig.add_trace(go.Scatter(x=df_planBTC[df_planBTC['Signal'] == 1.0].index, 
                         y=df_planBTC[df_planBTC['Signal'] == 1.0]['PriceUSD'],
                         name='Buy',
                         legendgroup = '1',
                         mode='markers',
                         marker=dict(
                            size=15, symbol='triangle-up', color='green')),
        secondary_y=True)
# Sell Signal
fig.add_trace(go.Scatter(x=df_planBTC[df_planBTC['Signal'] == -1.0].index, 
                         y=df_planBTC[df_planBTC['Signal'] == -1.0]['PriceUSD'],
                         name='Sell',
                         legendgroup = '1',
                         mode='markers',
                         marker=dict(
                            size=15, symbol='triangle-down', color='red')),
        secondary_y=True)

# Results
fig.add_trace(go.Scatter(x=df_planBTC.index,
                         y=bt_results._get_series(None).rebase()['buy_and_hold'],
                         #y=bt_results.prices['buy_and_hold'],
                         line=dict(color='blue', width=2),
                         name='Buy and Hold',
                         legendgroup = '2'), row=2, col=1)
fig.add_trace(go.Scatter(x=df_planBTC.index,
                         y=bt_results._get_series(None).rebase()['planBTC'],
                         #y=bt_results.prices['planBTC'],
                         line=dict(color='red', width=2),
                         name='PlanBTC',
                         legendgroup = '2'), row=2, col=1)

# Add figure title
fig.update_layout(
    title_text='PlanB@100TrillionUSD PlanBTC.com - Monthly Closing Data',
    width=1200, height=800,
    legend_tracegroupgap = 360,
    hovermode='x unified')

# Set y-axes titles
fig.update_yaxes(title_text='Price', type='log', secondary_y=True, row=1, col=1)
fig.update_yaxes(title_text='RSI', secondary_y=False, row=1, col=1)
fig.update_yaxes(title_text='Trading Results - Log', type='log', row=2, col=1)

fig.show()